# Tutorial : tensorflow2fluid转换VGG_16模型

VGG_16是CV领域的一个经典模型，本文档以tensorflow/models下的[VGG_16](https://github.com/tensorflow/models/blob/master/research/slim/nets/vgg.py)为例，展示如何将TensorFlow训练好的模型转换为PaddlePaddle模型。 
### 下载预训练模型

In [2]:
import urllib
import sys
def schedule(a, b, c):
    per = 100.0 * a * b / c
    per = int(per)
    sys.stderr.write("\rDownload percentage %.2f%%" % per)
    sys.stderr.flush()

url = "http://download.tensorflow.org/models/vgg_16_2016_08_28.tar.gz"
fetch = urllib.urlretrieve(url, "./vgg_16.tar.gz", schedule)

Download percentage 100.00%

### 解压下载的压缩文件

In [3]:
import tarfile
with tarfile.open("./vgg_16.tar.gz", "r:gz") as f:
    file_names = f.getnames()
    for file_name in file_names:
        f.extract(file_name, "./")

### 保存模型为checkpoint格式

tensorflow2fluid目前支持checkpoint格式的模型或者是将网络结构和参数序列化的pb格式模型，上面下载的`vgg_16.ckpt`仅仅存储了模型参数，因此我们需要重新加载参数，并将网络结构和参数一起保存为checkpoint模型

In [1]:
import tensorflow.contrib.slim as slim
from tensorflow.contrib.slim.nets import vgg
import tensorflow as tf
import numpy

with tf.Session() as sess:
    inputs = tf.placeholder(dtype=tf.float32, shape=[None, 224, 224, 3], name="inputs")
    logits, endpoint = vgg.vgg_16(inputs, num_classes=1000, is_training=False)
    load_model = slim.assign_from_checkpoint_fn("vgg_16.ckpt", slim.get_model_variables("vgg_16"))
    load_model(sess)
    
    numpy.random.seed(13)
    data = numpy.random.rand(5, 224, 224, 3)
    input_tensor = sess.graph.get_tensor_by_name("inputs:0")
    output_tensor = sess.graph.get_tensor_by_name("vgg_16/fc8/squeezed:0")
    result = sess.run([output_tensor], {input_tensor:data})
    numpy.save("tensorflow.npy", numpy.array(result))
    
    saver = tf.train.Saver()
    saver.save(sess, "./checkpoint/model")

INFO:tensorflow:Restoring parameters from vgg_16.ckpt


### 将模型转换为PaddlePaddle模型

注意：部分OP在转换时，需要将参数写入文件；或者是运行tensorflow模型进行infer，获取tensor值。两种情况下均会消耗一定的时间用于IO或计算，对于后一种情况，建议转换模型时将`use_cuda`参数设为`True`，加快infer速度

In [9]:
import tf2fluid.convert as convert
import argparse
parser = convert._get_parser()
parser.meta_file = "checkpoint/model.meta"
parser.ckpt_dir = "checkpoint"
parser.in_nodes = ["inputs"]
parser.input_shape = ["None,224,224,3"]
parser.output_nodes = ["vgg_16/fc8/squeezed"]
parser.use_cuda = "True"
parser.input_format = "NHWC"
parser.save_dir = "paddle_model"

convert.run(parser)

INFO:root:Loading tensorflow model...


INFO:tensorflow:Restoring parameters from checkpoint/model


INFO:tensorflow:Restoring parameters from checkpoint/model
INFO:root:Tensorflow model loaded!
INFO:root:TotalNum:86,TraslatedNum:1,CurrentNode:inputs
INFO:root:TotalNum:86,TraslatedNum:2,CurrentNode:vgg_16/conv1/conv1_1/weights
INFO:root:TotalNum:86,TraslatedNum:3,CurrentNode:vgg_16/conv1/conv1_1/biases
INFO:root:TotalNum:86,TraslatedNum:4,CurrentNode:vgg_16/conv1/conv1_2/weights
INFO:root:TotalNum:86,TraslatedNum:5,CurrentNode:vgg_16/conv1/conv1_2/biases
INFO:root:TotalNum:86,TraslatedNum:6,CurrentNode:vgg_16/conv2/conv2_1/weights
INFO:root:TotalNum:86,TraslatedNum:7,CurrentNode:vgg_16/conv2/conv2_1/biases
INFO:root:TotalNum:86,TraslatedNum:8,CurrentNode:vgg_16/conv2/conv2_2/weights
INFO:root:TotalNum:86,TraslatedNum:9,CurrentNode:vgg_16/conv2/conv2_2/biases
INFO:root:TotalNum:86,TraslatedNum:10,CurrentNode:vgg_16/conv3/conv3_1/weights
INFO:root:TotalNum:86,TraslatedNum:11,CurrentNode:vgg_16/conv3/conv3_1/biases
INFO:root:TotalNum:86,TraslatedNum:12,CurrentNode:vgg_16/conv3/conv3_2/we

### 加载转换后的PaddlePaddle模型，并进行预测
**需要注意，转换后的PaddlePaddle CV模型输入格式为NCHW**

In [1]:
import numpy
import tf2fluid.model_loader as ml

model = ml.ModelLoader("paddle_model", use_cuda=True)

numpy.random.seed(13)
data = numpy.random.rand(5, 224, 224, 3).astype("float32")
# NHWC -> NCHW
data = numpy.transpose(data, (0, 3, 1, 2))

results = model.inference(feed_dict={model.inputs[0]:data})

numpy.save("paddle.npy", numpy.array(results))

### 对比转换前后模型之前的预测结果diff

In [2]:
import numpy
paddle_result = numpy.load("paddle.npy")
tensorflow_result = numpy.load("tensorflow.npy")
diff = numpy.fabs(paddle_result - tensorflow_result)
print(numpy.max(diff))

3.33786e-06


### 需要注意的点
1. 转换后的模型需要注意输入格式，PaddlePaddle中输入格式需为NCHW格式  
2. 此例中不涉及到输入中间层，如卷积层的输出，需要了解的是PaddlePaddle中的卷积层输出，卷积核的`shape`与Tensorflow有差异  
3. 模型转换完后，检查转换前后模型的diff，在本例中，最大diff为3.34-e06，满足转换需求  